In [2]:
import random
import numpy as np
import pandas as pd
import time
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import operator
import time
import scipy as sp
import scipy.stats
import random
from sklearn import metrics
import implicit


In [3]:
#Loading the matrix
def load_matrix(filepath, num_users, num_items):
    counts = np.zeros((num_users, num_items))
    for i, line in enumerate(open(filepath, 'r')):
        user, item, count = line.strip().split(',')
        user = int(user)
        item = int(item)
        count = float(count)
        counts[user][item] = count
    return counts

def sparser(originalmatrix, num_users, num_items):
    counts = sparse.dok_matrix((num_users, num_items), dtype=float)
    for i in range(len(originalmatrix)):
        row = originalmatrix[i]
        for j in range(len(row)):
            if row[j] > 0:
                user = int(i)
                item = int(j)
                count = float(row[j])
                counts[user, item] = count
    counts = counts.tocsr()
    return counts

#Please enter the path to the input data
filepath = 'C:/Users/peter/Documents/uvt/implicit/finalxdata.csv'
matrix = load_matrix(filepath, num_users = 6518, num_items = 4036)


In [4]:
# Functions for the hiding of the user-item interactions for the validation and test set
def interactiontodrop(rownumber, row): 
    scoreditems = []
    items = dict(enumerate(row))
    for k in items:
        if items[k] > 0:
            scoreditems.append(k)
    random.seed(1)
    if not scoreditems:
        print(rownumber)
    return [rownumber, random.choice(scoreditems)]


def dropper(matrix, n):
    r = list(range(len(matrix)))
    random.seed(1)
    sample = random.sample(r,n)
    toodrop = []
    for x in sample:
        toodrop.append(interactiontodrop(x, matrix[x]))
    for item in toodrop:
        matrix[item[0]][item[1]] = 0
    return toodrop


In [5]:
#Creating the test set
test_set = dropper(matrix, 650)

In [11]:
#functions for the evaluation
def prediction(userid, similarities, matrix, originalmatrix, n=4036):
    predictions = np.dot(matrix[userid], similarities)
    predictions = predictions.toarray()
    predictions = predictions[0]
    for i in range(4036):
        if originalmatrix[userid][i] > 0:
            predictions[i] = -6000        
    dict = {key: value for (key, value) in (enumerate(predictions))}
    sorted_dict = sorted(dict.items(), key=operator.itemgetter(1), reverse=True)
    recommendation = []
    for i in range(n):
        recommendation.append(sorted_dict[i][0])
    return recommendation


def hlu_calc(dropped_items, coefficients, originalmatrix):
    model_hlus = []
    matrix, similarities = coefficients
    for b in dropped_items:
        ranking = (prediction(b[0], similarities, matrix, originalmatrix).index(b[1])+1)
        if ranking < 250:
            ind_hlu = 1 / (2 ** ((ranking) / 10))
            model_hlus.append(ind_hlu)
        else:
            model_hlus.append(0.0000000001)
    return model_hlus
    

In [7]:
#Building the model
sparsematrix = sparser(matrix.T, 4036, 6518)
model = implicit.nearest_neighbours.CosineRecommender(K=20)
model.fit(sparsematrix)
coefficients = [sparsematrix.T, model.similarity]

In [10]:
#Evaluating the model on the test set
model_hlus = hlu_calc(test_set, coefficients, matrix)
print("The models hlu is {}".format(np.average(model_hlus)))


The models hlu is 0.2847465711886884
